# Simple Trading Rule

In [1]:
from sysdata.sim.csv_futures_sim_data import csvFuturesSimData
import matplotlib.pyplot as plt
%matplotlib inline

Configuring sim logging


Work up a minimum example of a trend following system

Let's get some data

We can get data from various places; however for now we're going to use
prepackaged 'legacy' data stored in csv files

In [2]:
data = csvFuturesSimData()
data

csvFuturesSimData object with 239 instruments

We get stuff out of data with methods

In [3]:
print(data.get_instrument_list())
print(data.get_raw_price("EDOLLAR").tail(5))

['IRS', 'EUROSTX-SMALL', 'AEX_mini', 'VNKI', 'FTSEINDO', 'NASDAQ', 'GASOIL', 'EU-HOUSE', 'BRENT-LAST', 'US-PROPERTY', 'GASOILINE_ICE', 'CAD10', 'VIX', 'EU-OIL', 'EURO600', 'GILT', 'US-HEALTH', 'OATIES', 'BITCOIN', 'ALUMINIUM_LME', 'COPPER', 'COPPER-mini', 'NZD', 'GBP_micro', 'SP500_micro', 'BOBL', 'MILKWET', 'ETHER-micro', 'US20', 'SOYOIL', 'JPY_mini', 'GBPCHF', 'SUGAR11', 'GOLD-mini', 'EU-DJ-OIL', 'FTSETAIWAN', 'HANGENT_mini', 'HANG', 'MSCIEAFA', 'SOYMEAL', 'EURCAD', 'MSCIEMASIA', 'MILKDRY', 'AUD_micro', 'BRENT_W', 'WHEY', 'EUR', 'RUSSELL', 'BUXL', 'EU-CONSTRUCTION', 'WHEAT_mini', 'NOK', 'MSCIASIA', 'GAS_US', 'MILK', 'RUBBER', 'EU-CHEM', 'AUD', 'SP400', 'US-INDUSTRY', 'DJSTX-SMALL', 'ALUMINIUM', 'CNH', 'IRON', 'TWD-mini', 'DOW', 'COTTON2', 'GAS-LAST', 'LUMBER-new', 'SMI-MID', 'AEX', 'ETHANOL', 'KOSPI', 'EUROSTX-LARGE', 'US30', 'PALLAD', 'US5', 'CAD_micro', 'BTP', 'CORN', 'CAC', 'DAX', 'OMX', 'RICE', 'US-STAPLES', 'KR3', 'ZAR', 'GBPEUR', 'NIKKEI', 'HANG_mini', 'CHFJPY', 'RUR', 'EU-DJ-T

Exception: Instrument code EDOLLAR has no data!

data can also behave in a dict like manner (though it's not a dict)

In [ ]:
data['SP500']

In [ ]:
data.keys()

... however this will only access prices
(note these prices have already been backadjusted for rolls)

We have extra futures data here

In [ ]:
data.get_instrument_raw_carry_data("EDOLLAR").tail(6)

Technical note: csvFuturesSimData inherits from FuturesData which itself inherits from simData
The chain is 'data specific' <- 'asset class specific' <- 'generic'

Let's create a simple trading rule

No capping or scaling

In [ ]:
import pandas as pd
from sysquant.estimators.vol import robust_vol_calc


def calc_ewmac_forecast(price, Lfast, Lslow=None):
    """
    Calculate the ewmac trading rule forecast, given a price and EWMA speeds
    Lfast, Lslow and vol_lookback

    """
    # price: This is the stitched price series
    # We can't use the price of the contract we're trading, or the volatility
    # will be jumpy
    # And we'll miss out on the rolldown. See
    # https://qoppac.blogspot.com/2015/05/systems-building-futures-rolling.html

    price = price.resample("1B").last()

    if Lslow is None:
        Lslow = 4 * Lfast

    # We don't need to calculate the decay parameter, just use the span
    # directly
    fast_ewma = price.ewm(span=Lfast).mean()
    slow_ewma = price.ewm(span=Lslow).mean()
    raw_ewmac = fast_ewma - slow_ewma
    vol = robust_vol_calc(price.diff())
    return raw_ewmac / vol


Try it out

(this isn't properly scaled at this stage of course)


In [ ]:
instrument_code = 'EDOLLAR'
price = data.daily_prices(instrument_code)
ewmac = calc_ewmac_forecast(price, 32, 128)
ewmac.columns = ['forecast']
ewmac.tail(5)

In [ ]:
ewmac.plot();
plt.title('Forecast')
plt.ylabel('Position')
plt.xlabel('Time')


Did we make money?


In [ ]:
from systems.accounts.account_forecast import pandl_for_instrument_forecast
account = pandl_for_instrument_forecast(forecast=ewmac, price = price)
account.curve().plot();
plt.title('Profit and Loss')
plt.ylabel('PnL')
plt.xlabel('Time');

In [ ]:
account.percent.stats()